Generatively Pretrained Transformer (GPT)
https://arxiv.org/pdf/1706.03762

<h3> 1. Import Libraries </h3>

In [1]:
from platform import python_version

print(python_version())

3.8.10


In [2]:
import tensorflow as tf
import pandas as pd 
import requests
import tiktoken
import openai
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


<b> Check for available GPUs for Training </b> 

In [3]:
# Check for GPUs
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

<h3> 2. Import Data </h3> 

In [4]:
import urllib.request

def load_text_from_url(url):
    response = urllib.request.urlopen(url)
    text = response.read().decode('utf-8')
    return text

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
text = load_text_from_url(url)

<h3> 3. Basic Data Exploration </h3> 

In [5]:
print(f"The link of the dataset is {len(text)} chars")
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

The link of the dataset is 1115394 chars

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


<h3> 4. Create Basic Tokenization Functions and Encode Data </h3> 

<b> Encoding 1: Character Level </b> 

In [6]:
# Create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) } # This creates a lookup table for converting a character to an int
itos = { i:ch for i,ch in enumerate(chars) } # This creates a lookup table for converting an int to a char
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


<b> Encoding 2: Subword OpenAI GPT4 Dictionary 'cl100k_base'</b>

In [7]:
# open ai tokenizer function goes here (tiktoken)
#gpt2enc = tiktoken.get_encoding('gpt2')
# gpt4enc = tiktoken.get_encoding('cl100k_base')
#gpt2_vocab_size=gpt2enc.n_vocab
#print(f"The length of the gpt2 encoding dictionary is {gpt2_vocab_size}")
#print(gpt2enc.encode("Hello World"))
#print(gpt2enc.decode(gpt2enc.encode("Hello World")))

In [8]:
# Sentence piece from google

<h3> 5. Encode Dataset </h3>

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


<h3> 6. Create Training and Validation Datasets </h3> 

In [10]:
def train_validate_split(dataset,cutoff):
    n = int(cutoff*len(dataset))
    train_data = dataset[:n]
    val_data = dataset[n:]
    return train_data, val_data

In [11]:
cutoff = 0.90
train_data, val_data = train_validate_split(data,cutoff)

<h3> 7. Define Basic Bigram Transformer Model </h3> 

<b> Hyperparamters </b> 

In [12]:
# hyperparameters
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 50
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
patience = 3
plateau = 0.005

torch.manual_seed(1337)

<b> Data Batching Function </b> 

This function is used for generating a mini-batch of data from the specified dataset (training or validation) to be used during the training or evaluation process of a machine learning model. It randomly selects batch_size starting indices, then creates input sequences (x) and their corresponding target sequences (y) of length block_size. The input and target sequences are shifted by one position, making the model predict the next element in the sequence.

In [13]:
def get_batch(split):
    """
    This function generates a small batch of data consisting of inputs (x) and targets (y) from the training or validation dataset.
    
    Args:
    split (str): A string that indicates whether to use the training dataset or the validation dataset. It accepts two values, 'train' or 'val'.
    
    Returns:
    x (torch.Tensor): A tensor containing the input sequences for the mini-batch. The shape of the tensor is (batch_size, block_size).
    y (torch.Tensor): A tensor containing the target sequences for the mini-batch. The shape of the tensor is (batch_size, block_size).
    
    Global Variables:
    train_data (torch.Tensor): A tensor containing the training dataset.
    val_data (torch.Tensor): A tensor containing the validation dataset.
    batch_size (int): The number of samples in a single batch.
    block_size (int): The length of each input and target sequence.
    """
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

<b> Loss Estimation Function </b> 

In [14]:
@torch.no_grad() # tells pytorch we will not call "backward" on the function (back propagation)
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

<b> Simple Bigram Model </b> 

In [15]:
# Model copied from https://github.com/karpathy/nanoGPT
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [16]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


<h3> 6. Train Bigram Model </h3>

In [28]:
import time
import datetime
import pytz
import os

def train_model(filename, model, max_iters, eval_interval, plateau, patience, learning_rate):
    # Check if the trained model already exists
    if os.path.exists(filename):
        use_existing_model = input('Trained model already exists. Do you want to use it? (y/n) ')
        if use_existing_model.lower() == 'y':
            # Load the existing trained model
            model.load_state_dict(torch.load(filename))
            return model
        else:
            # Retrain the model
            os.remove(filename)
    
    # Train the model
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Create a list to store the losses
    loss_history = []

    best_val_loss = float('inf')
    num_epochs_no_improvement = 0

    start_time = time.time()
    start_datetime = datetime.datetime.now(pytz.timezone('US/Pacific'))

    fig, ax = plt.subplots()
    train_line, = ax.plot([], [], label='Train Loss')
    val_line, = ax.plot([], [], label='Validation Loss')
    ax.set_xlabel('Time')
    ax.set_ylabel('Loss')
    ax.set_title('Train and Test Loss over Time')
    ax.legend()

    est_completion_time = 'unknown'

    for iter in range(max_iters):

        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0:
            losses = estimate_loss()
            current_time = time.time()
            elapsed_time = current_time - start_time
            current_datetime = datetime.datetime.now(pytz.timezone('US/Pacific'))
            time_str = current_datetime.strftime('%Y-%m-%d %H:%M:%S %Z%z')

            # Store the loss every eval_interval iterations
            loss_history.append({'train': losses['train'], 'val': losses['val'], 'step': iter,
                                 'timestamp': current_datetime, 'elapsed_time': elapsed_time})

            # Check for plateau
            if best_val_loss - losses['val'] < plateau:
                num_epochs_no_improvement += 1
                if num_epochs_no_improvement >= patience:
                    print(f"Validation loss plateaued for {patience} epochs, stopping early...")
                    break
            else:
                best_val_loss = losses['val']
                num_epochs_no_improvement = 0

            # Update the plot with the new data
            train_line.set_data([l['timestamp'] for l in loss_history], [l['train'] for l in loss_history])
            val_line.set_data([l['timestamp'] for l in loss_history], [l['val'] for l in loss_history])
            ax.relim()
            ax.autoscale_view()
            fig.canvas.draw()

            if iter > 2 * eval_interval:
                time_elapsed = current_datetime - start_datetime
                time_per_iter = time_elapsed / iter
                est_remaining_iters = max_iters - iter
                est_remaining_time = est_remaining_iters * time_per_iter
                est_completion_time = (current_datetime + est_remaining_time).strftime('%Y-%m-%d %H:%M:%S %Z%z')
                print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, timestamp {time_str}, elapsed time {elapsed_time:.4f}s, est. completion {est_completion_time}")
            else:
                print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, timestamp {time_str}, elapsed time {elapsed_time:.4f}s")

        if iter == max_iters-1:
            print(f"Training completed: elapsed time {elapsed_time:.4f}s, estimated completion time {est_completion_time}")

    plt.show()
    
    # Save the trained model
    torch.save(model.state_dict(), filename)
    return model

In [30]:
#filename='shakespeare.pt'
filename='models/shakespeare.pt'
trained_model = train_model(filename, model, max_iters, eval_interval, plateau, patience, learning_rate)
trained_model

Trained model already exists. Do you want to use it? (y/n)  y


BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0): Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (1): Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (2): Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
   

In [33]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_tokens = trained_model.generate(context, max_new_tokens=1000)[0].tolist()
generated_text = decode(generated_tokens)
print(generated_text)

# Output the generated text to a file
with open('generated_text.txt', 'w') as f:
    f.write(generated_text)


Prave the sir, gretspair with off.
Good Let sir, A Wire you king; those new fair much
Is am plairy panient?

SICINIUS:
Is no from a call your lifkl'ma humber not,
Decould to Friend was reath your you llivest.

PRINIA:
If grate, a so, lear well surry them not mart;
So death his to upars that agarriys notly.

PEY:
But gone nother combume a love sond,
And regryous not them wreth, but.'

COMPEY:
I not my emayer agaid weell in thand laudes,
Be on of Lood that falst things on that eyelf,
Her, whrings I valliness.

Second LAdMP:
R; ares
you but shatorns fid a clong bidscal on yourse toden ranging.

PETEN XIO:

That allies now them honour death of the man' tough ime.
Thou not murnt and dear; abod; he thou want thy sit thy sI
Contame in a lance,---

GAREn:
Gend, well very to head a her: his dubrath like him:
And this blothe! I never lord,
And as the lost impan'ding the reath?

SINIUS:
Whos nord.

KING EDWARD IV:
Here is me, he gret.

HENRY CABEL:
Bricklacil's devargel 's is halence man, yet th